# Segmenting and Clustering Neighborhoods in Toronto - Assignment

#### Import the libraries to prepare the data

In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

#### Create a soup objected from the wikipedia link containing the postal codes

In [15]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url).text
soup = BeautifulSoup(html_data,"html5lib") 

#### Extract the data from the table and create a dictionary

In [16]:
table = soup.find("table")
df_content = []
for row in table.find_all("td") :
    cell = {}
    
    if row.span.text=='Not assigned':
        pass
    
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        df_content.append(cell)

#### Use the dictionary to create a data frame
##### Also, correct some of the names and join neighborhoods with the same postal code

In [17]:
df_column_names=['PostalCode', 'Borough', 'Neighborhood']
neighbourhoods_df=pd.DataFrame(data=df_content, columns=df_column_names)
neighbourhoods_df['Borough']=neighbourhoods_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


#### Show the shape of the resulting data frame

In [18]:
neighbourhoods_df.shape

(103, 3)

In [19]:
neighbourhoods_df[:11]

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Get the coordinates and append them to the data frame

In [20]:
coordinates_url="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv"
coordinates_df=pd.read_csv(coordinates_url)

neighbourhoods_df['Latitude'] = neighbourhoods_df.PostalCode.map(coordinates_df.set_index('Postal Code')['Latitude'].to_dict())
neighbourhoods_df['Longitude'] = neighbourhoods_df.PostalCode.map(coordinates_df.set_index('Postal Code')['Longitude'].to_dict())

In [21]:
neighbourhoods_df.shape

(103, 5)

In [22]:
neighbourhoods_df[:11]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


#### Define Foursquare Credentials

In [30]:
# @hidden_cell
CLIENT_ID = 'ZZBEKKTJGWTU0HNGJ2ROO5JDHSKAXVABHUJKBBYZG1WNWQ0W' # your Foursquare ID
CLIENT_SECRET = 'JS1SDJDRRFBWX0SXVSS5GPCXVG4J3BAKY0AWJMLE2IJH4OFG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

#### Create a function that finds venues for each neighbourhood

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
Toronto_venues = getNearbyVenues(names=neighbourhoods_df['Neighborhood'],
                                   latitudes=neighbourhoods_df['Latitude'],
                                   longitudes=neighbourhoods_df['Longitude'])


Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [ ]:
print(Toronto_venues.shape)
Toronto_venues.head()

#### Analyze each neighbourhood and prepare for K means clustering

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.insert(loc=0, column='Neighborhood name', value=Toronto_venues['Neighborhood'].values.tolist()) 
toronto_onehot.head()

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood name').mean().reset_index()
toronto_grouped.head()

#### Apply K-means clustering

In [ ]:
from sklearn.cluster import KMeans
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood name', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

#### Merge into one data frame for later visualization

In [ ]:
# Not all the neighborhoods were analyzed because of the API limit of 100
print(len(set(toronto_grouped['Neighborhood name'])))
print(len(set(neighbourhoods_df['Neighborhood'])))

In [ ]:
# Remove the neighborhoods that are not present in the analysis because of the API limit
for element in neighbourhoods_df['Neighborhood']:
    if element in set(toronto_grouped['Neighborhood name']):
        pass
        
    else:
        print(element)
        neighbourhoods_df = neighbourhoods_df[neighbourhoods_df.Neighborhood != element]

print(len(set(neighbourhoods_df['Neighborhood'])))

In [ ]:
# Insert cluster labels in the grouped data frame to match with the neighborhood name
toronto_grouped.insert(1, 'Cluster Labels', kmeans.labels_)

In [ ]:
# merge postal code, borough, neighborhood name, latitude, longitude, and cluster label
toronto_merged = neighbourhoods_df
toronto_merged = toronto_merged.join(toronto_grouped.iloc[:,0:2].set_index('Neighborhood name'), on='Neighborhood')
toronto_merged.head()

#### Visualize with folium

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors
!conda install -c conda-forge folium=0.5.0 --yes
import folium

In [ ]:
# create map. The latitude and longitude of toronto were found on wikipedia
map_clusters = folium.Map(location=[43.651070,  -79.347015], zoom_start=10.4)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters